In [45]:
import pandas as pd
path_to_results = '/home/pestefo/projects/ra_recommendator_conrec/results/'
experiment = '20190717_1108_5p'
scenarios = ['A', 'B', 'C', 'D']
data = {}
for s in scenarios:
    data[s] = pd.read_csv(path_to_results+experiment+'/recall_for_scenario_'+s+'.csv', sep='\t', index_col=0)

In [66]:
import plotly.graph_objects as go


scenario = 'A'
up_to_top100 = data[scenario].columns[:-10]

for scenario in scenarios:
    fig = go.Figure()
    for col in data[scenario][up_to_top100]:
        fig.add_trace(go.Box(
            y=df[col],
            name=col,
            boxpoints='outliers', # only outliers
            marker_color='rgb(9,56,125)',
            line_color='rgb(9,56,125)'
        ))

    fig.update_layout(
        title_text="Recall Scenario "+scenario+" - 5 participants, 100 questions",
        showlegend=False,
        height=500)
    
    fig.show()
    

In [ ]:
# all questions in which any of the scenarios performed poorly, ie. Recall@60 <= 0.2
questions_with_bad_performance = [9135, 9197, 9251, 9273, 9277, 9427, 9606, 9615, 9620, 9653, 9684, 9750, 9796, 10006, 10013, 10020, 10027, 10072, 10105, 10120, 10149, 10238, 10280, 10284, 10313, 10323, 10342]


In [67]:
"""
TODO:
- Subplots para los boxplot
- Exportar para que se vean en el README.md
- Tabla de las flechas
"""

'\nTODO:\n- Subplots para los boxplot\n- Exportar para que se vean en el README.md\n- Tabla de las flechas\n'